In [2]:
import librosa
import numpy as np
import IPython.display as ipd
import os
import pandas as pd

## Preparing data

#### Preparing start samples

In [8]:
X_start = []
paths = librosa.util.find_files(r'orig\Без шума\Старт')
for i in paths:
    x, sr = librosa.load(i)
    mfcc = librosa.feature.mfcc(y=x, sr=sr, hop_length=512, n_mfcc=20)
    mfcc = np.mean(mfcc, axis=1)
    X_start.append(mfcc)
y_start = np.ones(np.shape(X_start)[0])
np.shape(X_start)

(71, 20)

#### Preparing other samples

In [9]:
X_other = []
voices_path = r'audiobooks\data\ru_open_stt\private_buriy_audiobooks_2\0'
for path in os.listdir(voices_path):
    paths = librosa.util.find_files(voices_path + '\\' + path)
    for file in paths:
        x, sr = librosa.load(file)
        mfcc = librosa.feature.mfcc(y=x, sr=sr, hop_length=512, n_mfcc=20)
        mfcc = np.mean(mfcc, axis=1)
        X_other.append(mfcc)
y_other = np.zeros(np.shape(X_other)[0])
np.shape(X_other)

(3359, 20)

## Loading data

#### Loading start samples

In [ ]:
DF_start = pd.read_csv(r'data\\start.csv', index_col=0)
DF_y_start = pd.read_csv(r'data\\start_y.csv', index_col=0)

#### Loading other samples

In [ ]:
DF_other = pd.read_csv(r'data\\other.csv', index_col=0)
DF_y_other = pd.read_csv(r'data\\other_y.csv', index_col=0)

## Converting data to dataframe

#### Converting start samples

In [17]:
DF_start = pd.DataFrame(X_start, columns=list('mfcc' + " " + str(i) for i in range(1, 21)))
DF_y_start = pd.DataFrame(y_start, columns=['y'])

#### Converting other samples

In [20]:
DF_other = pd.DataFrame(X_other, columns=list('mfcc' + " " + str(i) for i in range(1, 21)))
DF_y_other = pd.DataFrame(y_other, columns=['y'])

## Merging samples

In [22]:
DF_samples = DF_start.append(DF_other, ignore_index=True)
DF_y = DF_y_start.append(DF_y_other, ignore_index=True)

## Saving samples

#### Saving start samples

In [13]:
DF_start.to_csv(r"data\\start.csv")
DF_y_start.to_csv(r"data\\start_y.csv")

#### Saving other samples

In [14]:
DF_other.to_csv(r"data\\other.csv")
DF_y_other.to_csv(r"data\\other_y.csv")

#### Saving merged samples

In [15]:
DF_samples.to_csv(r"data\\samples.csv")
DF_y.to_csv(r"data\\samples_y.csv")